In [5]:
import pandas as pd
from opencage.geocoder import OpenCageGeocode
import folium
from folium.plugins import FastMarkerCluster
import matplotlib.pyplot as plt

In [6]:
data=pd.read_csv('C:/Users/anranzheng/Dropbox (UFL)/Gap analysis/data/rt GTFS data/stop.csv')
data.head(10)

,OBJECTID,Id,route,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,bus_num
0,1,0,3,1468,1468,N Main Post Office,Westbound NW 14th Ave @ Farside Main Street,29.664846,-82.325668,1
1,2,0,8,446,446,UF Health Archer Road,Westbound Archer RD @ Farside SW 16th ST,29.639050,-82.344089,1
2,3,0,9,447,447,Hunters Run,Northbound SW 25th TER @ Farside Williston RD,29.612789,-82.359331,2
3,5,0,15,255,255,NW 13th Street,Northbound NW 13th ST @ Farside NW 26th PL,29.677464,-82.338961,1
4,6,0,16,782,782,Sugar Hill,Northbound SE 20th PL @ Nearside SE 4th ST,29.632672,-82.315935,1
5,7,0,21,1236,1236,Cabana Beach,Northbound SW 52nd BLVD @ Farside SW 20th AVE,29.635913,-82.397884,2
6,8,0,34,951,951,Southbound SW 26th Drive @ SW 40th Place,Southbound SW 26th DR @ Nearside SW 40th PL,29.614132,-82.361212,1
7,9,0,35,945,945,Ridgemar Commons,Eastbound SW 35th PL @ Farside SW 32nd TER,29.621566,-82.370416,2
8,10,0,38,1337,1337,Gainesville Place,Southbound Gainesville Place Apartments @ Near...,29.621832,-82.365480,3
9,12,0,43,660,660,Shands Hospital @ Helipad,Westbound Shealy DR @ Farside SW 16TH AVE,29.635502,-82.351244,1


In [7]:
lats=data['stop_lat'].to_list()
lons=data['stop_lon'].to_list()

In [13]:
m= folium.Map(
    location=[29.635502,-82.351244],
    zoom_start=13,tiles='Stamen Toner')
#FastMarkerCluster(data[['latitudes', 'longitudes']].values.tolist()).add_to(m)
folium.LayerControl().add_to(m)

for row in data.iterrows():
    row=row[1]
    folium.CircleMarker(location=(row["stop_lat"],
                                  row["stop_lon"]),
                        radius= row['bus_num'],
                        color="#007849",
                        popup=row['stop_name']).add_to(m)


# save the map 
#m.save("save_file.html")
    
display(m)


#FastMarkerCluster(data=list(zip(lats,lons))).add_to(m)
#folium.LayerControl().add_to(m)

In [ ]:
import plotly.graph_objects as go

data['text'] = data['Facility_Name'] +'Total Cases:'+ data['Total'].astype(str)

fig = go.Figure(data=go.Scattergeo(
        locationmode = 'USA-states',
        lon = data['longitudes'],
        lat = data['latitudes'],
        text = data['text'],
        mode = 'markers',
        marker = dict(
            size = data['Total']/5000,
            opacity = 0.8,
            reversescale=True,
            autocolorscale = False,
            symbol = 'circle',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'Blues',
            cmin = 0,
            color = data['Total'],
            cmax = data['Total'].max(),
            colorbar_title="Total Covid 19 cases<br>count per bounty"
        )))

fig.update_layout(
        title = 'Hospitals location in USA<br>(Hover for Hospital names)',
        geo = dict(
            scope='usa',
            projection_type='albers usa',
            showland = True
        ),
    )
fig.show()

In [14]:
import random
import numpy as np
import pandas as pd 
import scipy.spatial
from haversine import haversine
def distance(p1,p2):
  return haversine(p1[1:],p2[1:])
def cluster_centroids(data, clusters, k):
  results=[]
  for i in range(k):
    results.append(np.average(data[clusters == i],weights=np.squeeze(np.asarray(data[clusters == i][:,[0]])),axis=0))
  return results
def kmeans(data, k=None, centroids=None, steps=30):
  # Forgy initialization method: choose k data points randomly.
  centroids = data[np.random.choice(np.arange(len(data)), k, False)]
  elbow = []
  for _ in range(max(steps, 1)):
    sqdists = scipy.spatial.distance.cdist(centroids, data, lambda u, v: (distance(u,v)**2))
    elbow.append(sqdists)
    #sse=np.sum((np.mean(elbow)-elbow)**2)
    # Index of the closest centroid to each data point.
    clusters = np.argmin(sqdists, axis=0)
    new_centroids = cluster_centroids(data, clusters, k)
    if np.array_equal(new_centroids, centroids):
      break
    centroids = new_centroids
  
  return clusters, centroids,elbow

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [18]:
vals = data[['bus_num','stop_lat','stop_lon']].values
sse=[]
for k in range(1,10):
    clusters,centroids,elbow=kmeans(vals,k)
    sqdist = np.sum((elbow-np.mean(elbow))**2)
    sse.append(sqdist)
sse=sorted(sse,reverse=True)

KeyError: ('bus_num', 'stop_lat', 'stop_lon')

In [17]:
vals

array([[  1.        ,  29.66484569, -82.32566755],
       [  1.        ,  29.63904961, -82.34408885],
       [  2.        ,  29.61278861, -82.35933118],
       [  1.        ,  29.67746376, -82.33896085],
       [  1.        ,  29.63267161, -82.31593452],
       [  2.        ,  29.63591311, -82.39788367],
       [  1.        ,  29.61413182, -82.36121169],
       [  2.        ,  29.62156578, -82.37041567],
       [  3.        ,  29.62183194, -82.36547951],
       [  1.        ,  29.63550244, -82.35124418],
       [  1.        ,  29.6283902 , -82.4363982 ],
       [  2.        ,  29.63673627, -82.36795351],
       [  1.        ,  29.63941661, -82.37065234],
       [  1.        ,  29.64058   , -82.36155   ],
       [  1.        ,  29.63158944, -82.35143001],
       [  1.        ,  29.65797763, -82.33714947],
       [  1.        ,  29.60719585, -82.43879881],
       [ 12.        ,  29.62515397, -82.38788979],
       [ 14.        ,  29.64556734, -82.32269681],
       [  5.        ,  29.67962